In [1]:
import os
import sys
import torch as t
from torch import Tensor
import einops
from ipywidgets import interact
import plotly.express as px
from ipywidgets import interact
from pathlib import Path
from IPython.display import display
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
import typeguard

# Make sure exercises are in the path
chapter = r"chapter0_fundamentals"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = exercises_dir / "part1_ray_tracing"
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from plotly_utils import imshow
from part1_ray_tracing.utils import render_lines_with_plotly, setup_widget_fig_ray, setup_widget_fig_triangle
import part1_ray_tracing.tests as tests

MAIN = __name__ == "__main__"

In [27]:
def make_rays_1d(num_pixels: int, y_limit: float) -> t.Tensor:
    '''
    num_pixels: The number of pixels in the y dimension. Since there is one ray per pixel, this is also the number of rays.
    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both endpoints.

    Returns: shape (num_pixels, num_points=2, num_dim=3) where the num_points dimension contains (origin, direction) and the num_dim dimension contains xyz.

    Example of make_rays_1d(9, 1.0): [
        [[0, 0, 0], [1, -1.0, 0]],
        [[0, 0, 0], [1, -0.75, 0]],
        [[0, 0, 0], [1, -0.5, 0]],
        ...
        [[0, 0, 0], [1, 0.75, 0]],
        [[0, 0, 0], [1, 1, 0]],
    ]
    '''
    origin = t.Tensor([0, 0, 0])
    middle = t.tensor([1, 1, 0])
    pair = t.stack([origin, middle])

    rays = einops.repeat(pair, 'p c -> num_rays p c', num_rays=num_pixels).clone()

    t.linspace(-y_limit, y_limit, num_pixels, out=rays[:, 1, 1])

    return rays

    

rays1d = make_rays_1d(9, 10.0)

fig = render_lines_with_plotly(rays1d)

In [31]:
fig = setup_widget_fig_ray()
display(fig)

@interact
def response(seed=(0, 10, 1), v=(-2.0, 2.0, 0.01)):
    t.manual_seed(seed)
    L_1, L_2 = t.rand(2, 2)
    P = lambda v: L_1 + v * (L_2 - L_1)
    x, y = zip(P(-2), P(2))
    with fig.batch_update(): 
        fig.data[0].update({"x": x, "y": y}) 
        fig.data[1].update({"x": [L_1[0], L_2[0]], "y": [L_1[1], L_2[1]]}) 
        fig.data[2].update({"x": [P(v)[0]], "y": [P(v)[1]]})

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '3d3fed34-3e9a-468f-8841-709eadf18527', 'x': [], 'y': []},
             {'marker': {'size': 12},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'eba638d7-f1ed-43b4-93d5-f21a64da2b44',
              'x': [],
              'y': []},
             {'marker': {'size': 12, 'symbol': 'x'},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '131aa812-8211-407f-9514-0956e62a7616',
              'x': [],
              'y': []}],
    'layout': {'height': 500,
               'showlegend': False,
               'template': '...',
               'width': 600,
               'xaxis': {'range': [-1.5, 2.5]},
               'yaxis': {'range': [-1.5, 2.5]}}
})

interactive(children=(IntSlider(value=5, description='seed', max=10), FloatSlider(value=0.0, description='v', …

In [32]:
segments = t.tensor([
    [[1.0, -12.0, 0.0], [1, -6.0, 0.0]], 
    [[0.5, 0.1, 0.0], [0.5, 1.15, 0.0]], 
    [[2, 12.0, 0.0], [2, 21.0, 0.0]]
])

In [35]:
segments[0] # intersects with ray to -10 and to -7.5

tensor([[  1., -12.,   0.],
        [  1.,  -6.,   0.]])

In [36]:
segments[1] # intersects with nothing

tensor([[0.5000, 0.1000, 0.0000],
        [0.5000, 1.1500, 0.0000]])

In [38]:
segments[2] # intersects with y = 10 and y=7.5 

tensor([[ 2., 12.,  0.],
        [ 2., 21.,  0.]])

In [40]:
rays1d * 2

tensor([[[  0.,   0.,   0.],
         [  2., -20.,   0.]],

        [[  0.,   0.,   0.],
         [  2., -15.,   0.]],

        [[  0.,   0.,   0.],
         [  2., -10.,   0.]],

        [[  0.,   0.,   0.],
         [  2.,  -5.,   0.]],

        [[  0.,   0.,   0.],
         [  2.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  2.,   5.,   0.]],

        [[  0.,   0.,   0.],
         [  2.,  10.,   0.]],

        [[  0.,   0.,   0.],
         [  2.,  15.,   0.]],

        [[  0.,   0.,   0.],
         [  2.,  20.,   0.]]])

In [41]:
render_lines_with_plotly(t.concatenate([rays1d * 2, segments], dim=0))

In [70]:
@jaxtyped(typechecker=typeguard.typechecked)
def intersect_ray_1d(ray: Float[t.Tensor, "n=2 d=3"], segment: Float[t.Tensor, "n d"]) -> bool:
    '''
    ray: shape (n_points=2, n_dim=3)  # O, D points
    segment: shape (n_points=2, n_dim=3)  # L_1, L_2 points

    Return True if the ray intersects the segment.
    '''
    O = ray[0, 0:2]
    D = ray[1, 0:2] # shape (2,)
    L1 = segment[0, 0:2]
    L2 = segment[1, 0:2]

    mat = t.stack([D, L1 - L2], dim=1)

    rhs = L1 - O
    try:
        uv = t.linalg.solve(mat, rhs)
    except Exception as e:
        return False
    u, v = uv[0].item(), uv[1].item()
    return (u >= 0) and (0 <= v <= 1)




tests.test_intersect_ray_1d(intersect_ray_1d)
tests.test_intersect_ray_1d_special_case(intersect_ray_1d)

All tests in `test_intersect_ray_1d` passed!
All tests in `test_intersect_ray_1d_special_case` passed!


In [46]:
t.linalg.solve(2*t.eye(3), t.Tensor([2,3,4]))

tensor([1.0000, 1.5000, 2.0000])

In [ ]:
def intersect_rays_1d(rays: Float[Tensor, "nrays 2 3"], segments: Float[Tensor, "nsegments 2 3"]) -> Bool[Tensor, "nrays"]:
    '''
    For each ray, return True if it intersects any segment.
    '''
    nrays = rays.shape[0]
    nsegments = segments.shape[0]

    Os = rays[:, 0, :2] # (nrays, 2dim)
    Ds = rays[:, 1, :2]
    L1s = segments[:, 0, :2] # (nsegments, 2dim)
    L2s = segments[:, 1, :2]

    # Repeat each ray for all segments - one batch row for one ray
    batch_Ds = einops.repeat(Ds, 'nrays 2dim -> nrays nsegments 2dim', nsegments=nsegments)
    batch_Os = einops.repeat(Os, 'nrays 2dim -> nrays nsegments 2dim', nsegments=nsegments)

    # Repeat each segment for all rays one batch column for one ray
    batch_L1s = einops.repeat(L1s, 'nsegments 2dim -> nrays nsegments 2dim', nrays=nrays)
    batch_L2s = einops.repeat(L2s, 'nsegments 2dim -> nrays nsegments 2dim', nrays=nrays)

    mat_columns = [Ds, L1s - L2s]
    mats = einops.rearrange(mat_columns, 'cols nrays nsegments dim -> nrays nsegments dim cols')

    rhss = batch_L1s - batch_Os # shape (nrays, nsegments, 2dim)

    # TODO check for invertibility separately

    solutions = t.linalg.solve(mats, rhss) # shape(nrays, 2dim)

    us = solutions[..., 0]
    vs = solutions[..., 1]

    # TODO this is probably wrong
    return (us >= 0).any(dim=1) & (0 <= vs <= 1).any(dim=1)




tests.test_intersect_rays_1d(intersect_rays_1d)
tests.test_intersect_rays_1d_special_case(intersect_rays_1d)